In [1]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
from secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('sp_500_stocks.csv')

In [4]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 94201521,
 'calculationPrice': 'close',
 'change': 0.89,
 'changePercent': 0.00611,
 'close': 0,
 'closeSource': 'cliffioa',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': 'lsCoe',
 'highTime': 1739200190107,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 151.53,
 'iexCloseTime': 1709221862210,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 150.984,
 'iexOpenTime': 1741832834926,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1686726702823,
 'latestPrice': 152.35,
 'latestSource': 'Close',
 'latestTime': 'November 23, 2022',
 'latestUpdate': 1704788425634,
 'latestVolume': None,
 'low': 0,
 'lowSource': 'Coels',
 'lowTime': 1732298584210,
 'marketCap':

In [5]:
my_columns = ['Ticker', 'Price', 'P/E ratio', 'Number of shares to buy']

def chunks(lst, n):
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_appi_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_appi_call_url).json()
    
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['quote']['peRatio'],
                        'N/A'
                        ],
                    index=my_columns
                    ),
                ignore_index=True
                )
        except:
            continue
        
final_dataframe

,Ticker,Price,P/E ratio,Number of shares to buy
0,A,160.77,35.48,N/A
1,AAL,14.72,-6.02,N/A
2,AAP,157.81,17.99,N/A
3,AAPL,156.84,25.41,N/A
4,ABBV,160.31,21.41,N/A
...,...,...,...,...
496,YUM,128.57,29.02,N/A
497,ZBH,119.20,90.33,N/A
498,ZBRA,276.14,30.54,N/A
499,ZION,53.67,6.02,N/A


In [9]:
final_dataframe.sort_values(by='P/E ratio', ascending=True)


,Ticker,Price,P/E ratio,Number of shares to buy
418,STE,185.44,-2320.23,N/A
95,CINF,113.40,-604.33,N/A
471,VTR,46.61,-427.83,N/A
446,TWTR,56.30,-372.6,N/A
445,TTWO,106.32,-133.39,N/A
...,...,...,...,...
203,GOOG,102.76,None,N/A
252,J,123.90,None,N/A
346,NWS,19.03,None,N/A
440,TROW,127.50,None,N/A


In [20]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Price,P/E ratio,Number of shares to buy
40,AON,316.97,None,N/A
71,BRK.B,327.04,None,N/A
135,DISCK,25.38,None,N/A
189,FOX,29.87,None,N/A
192,FRT,109.54,None,N/A
203,GOOG,102.76,None,N/A
252,J,123.90,None,N/A
346,NWS,19.03,None,N/A
440,TROW,127.50,None,N/A
450,UA,8.84,None,N/A
